# Lab5: Hyper parameter tuning using Grid, Random, and Halving searches

In this lab we apply the hyper parameter search strategies we learnt in class. We continue with the prediction task studied in Lab 4.

**Task**: _Predict if a credit card account will default._

This is a binary classification problem. We'll load data and build the pipeline for a support vector machine (SVM). SVM is an effective technique that can correctly clasify on difficult datasets, but it has multiple hyper-parameters that require careful tuning. It also slows down when we have lots of training data. Thus, SVM allows us to see the benefit of standard Grid/Random search vs their Halving counterparts (which are smarter about which hyperparameters to evaluate thoroughly).


The outline of this lab is as follows:

1. Load data and split it into training and test.
1. Construct the pipeline with preprocessing and SVM classifier
1. Use grid search strategy to search for two hyper parameters
  1. Use **Halving** grid search to speed up
1. Use random search strategy to search for best hyper parameter values
  1. Use **Halving** random search to speed up
1. Exercise to try a similar exercise with k-nearest neighbor.

### Prepare data

Let's fetch the data and load it:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load the Default.csv dataset
data_url = 'https://drive.google.com/uc?export=download&id=13bQQfKDjiuBA1njsvCEW81c4Q2huRE9z'
data = pd.read_csv(data_url)
print('High level data description:')
data.info()
print('\nFirst 3 rows:')
data.head(3)


High level data description:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB

First 3 rows:


,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947


The data attributes have the following meaning:

* **default**: A categorical variable with levels No and Yes indicating whether the customer defaulted on their debt
* **student**: A categorical variable with levels No and Yes indicating whether the customer is a student
* **balance**: The average balance that the customer has remaining on their credit card after making their monthly payment
* **income**: Income of customer

Before doing much, let's set aside a portion of the data for testing: we'll eventually want to estimate the test-error for whatever model we choose. Separating the data sets now will avoid any potential leakage of information from test data to training.



In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop("default", axis=1)
y = data["default"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


Separate the training data into X matrix and y columns — the format expected by most scikit learn predictive models.

### Build the pipeline

The preprocessing steps are similar to what we had before. We add a new classifier, `SVC`, to the end to construct a full pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC   # Support Vector Classifier
from sklearn import set_config

set_config(display='diagram') # shows the pipeline graphically when printed

cat_attribs = ["student"]
num_attribs = ["balance", "income"]

preprocess_pipeline = ColumnTransformer([ # handle each type of column with appropriate pipeline
        ("cat", OneHotEncoder(drop="first"), cat_attribs),
        ("num", StandardScaler(), num_attribs),
    ])

svm_pipeline = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('svm', SVC()),
])

svm_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['student']),
                                                 ('num', StandardScaler(),
                                                  ['balance', 'income'])])),
                ('svm', SVC())])

### Setup the grid and search

(*Optional description of hyper parameters of SVM.*)
SVM can be thought of as drawing class boundaries to separate positives and negatives with some margin around it (margin reduces the chance that the boundary will be violated by data). Prediction is made by (similarity) weighted combination of votes of neighbors.

SVM has two important hyperparameters:
1. `C` $∈ [0, ∞)$ a continuous variable that represents the cost misclassification (crossing the boundary). For high values, the separating boundary will be wiggly and complex.
1. `gamma` $∈ [0, ∞)$ controls how rapidly similarity drops off with distance.

High values of `C` and `gamma` can lead to overfitting. In contrast, too low values can underfit.

Let's search for a good value of `C` and `gamma` for this dataset using various strategies we have learnt, starting with Grid search:

In [ ]:
param_grid = [ # Important: '__' separates step from their hyperparameters in key names below
    {'svm__C': np.logspace(-3, 3, 4),     # log(C) ranges from -3 to 3 in 4 steps.
     'svm__gamma': np.logspace(-2, 2, 5)  # log(gamma) ranges from -2 to 2 in 5 steps.
     }
    ]
# Check what's in this parameter grid
print('The parameter grid : ')
print(param_grid)

# Let's compare all 20 combinations of these values using cross validation:
#   Divide into K parts, set aside one for testing, fit and test SVC's at all possible combinations
#   Rotate folds and do so K times, average the results.
grid_search = GridSearchCV(svm_pipeline, param_grid, cv=3, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)
print('\n\nThe best parameters are ', grid_search.best_params_)

grid_cv_res = pd.DataFrame(grid_search.cv_results_) # convert to DF for convenience
grid_cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)  # sort the data frame
# select only the columns that start with 'param_' and the column 'mean_test_score'
grid_cv_res.filter(regex = '(^param_|mean_test_score)', axis=1).head()

The parameter grid : 
[{'svm__C': array([1.e-03, 1.e-01, 1.e+01, 1.e+03]), 'svm__gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])}]


The best parameters are  {'svm__C': np.float64(1000.0), 'svm__gamma': np.float64(1.0)}


,param_svm__C,param_svm__gamma,mean_test_score
17,1000.0,1.0,0.628530
13,10.0,10.0,0.628117
12,10.0,1.0,0.624655
18,1000.0,10.0,0.621488
16,1000.0,0.1,0.614519


Took a while to fit, didn't it? Now let's use the Halving version that evaluates all the same parameter combinations, but does so cheaply (using less data) first, then narrows down on the more promising ones for fuller evaluation (using more data).


Let's see if Halving version of it saves us any time...

In [ ]:
from sklearn.experimental import enable_halving_search_cv # needed to enable Halving features
from sklearn.model_selection import HalvingGridSearchCV

param_grid = [
    {'svm__C': np.logspace(-3, 3, 4),
     'svm__gamma': np.logspace(-2, 2, 5)
     },
    ]
# Check what's in this parameter grid
print('The parameter grid : ')
print(param_grid)

# Change to Halving strategy starting here
halving_grid_search = HalvingGridSearchCV(svm_pipeline, param_grid, cv=3,
                                    min_resources='exhaust', # use all data in the last round, back calculate to determine how much to start with
                                    scoring='balanced_accuracy')
halving_grid_search.fit(X_train, y_train)
print('The best parameters are ', halving_grid_search.best_params_)

halving_grid_cv_res = pd.DataFrame(halving_grid_search.cv_results_)  # convert to DF for convenience
# In the end, we care about performances in the last iteration (using most data)
# So, let's sort by iteration (descending), then by test score (descending)
halving_grid_cv_res.sort_values(by=['iter', 'mean_test_score'], ascending=False, inplace=True)
# and check the top few rows
halving_grid_cv_res.filter(regex = '(iter|^param_|mean_test_score|n_resources)', axis=1).head(30)


The parameter grid : 
[{'svm__C': array([1.e-03, 1.e-01, 1.e+01, 1.e+03]), 'svm__gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])}]
The best parameters are  {'svm__C': np.float64(1000.0), 'svm__gamma': np.float64(1.0)}


,iter,n_resources,param_svm__C,param_svm__gamma,mean_test_score
29,2,7497,1000.000,1.00,0.628529
27,2,7497,10.000,10.00,0.628116
28,2,7497,10.000,1.00,0.624654
24,1,2499,1000.000,1.00,0.603327
23,1,2499,10.000,1.00,0.590076
26,1,2499,10.000,10.00,0.586557
22,1,2499,1000.000,10.00,0.584085
25,1,2499,1000.000,0.10,0.576877
20,1,2499,1000.000,0.01,0.529144
21,1,2499,10.000,0.10,0.528730


Your Advantage May Vary, but you might have seen some speed up, even with only 20 parameters to evaluate. It gets better when you have more combinations to evaluate.

Three points:
1. Don't read much into the difference in test_scores at the same hyper-parameter values across regular vs halving grid: they are due to use of different data subsets.
1. With the halving search, in iterations before the last, you may see higher test scores, even for different hyper-parameter combinations. We don't select hyper-parameters based on those rounds: they used significantly less data, hence less reliable.
1. Although, it's called 'halving', scikit learn uses a factor of three by default, as you can see from the `n_resources` across iterations. Typically scikit learn chooses good defaults.

***
**Try (when you have some spare time, since it'll take a little while to run) to better appreciate the benefits of halving search**: For both regular Grid search and for the Halving version, increase the number of steps within logspace by one for each hyper-parameter.
***

### Random search strategy


In the previous two blocks we compared Grid search with its Halving version. In either case, we precommitted to **THE GRID** of parameters. Now, let's explore their random version. Recall, in each iteration the Randomized search draws a new random value for *each* hyper-parameter. Therefore, each iteration explores a potentially new value of each parameter. But to do that we need to supply the **distributions to sample those from**.

Both randomized search and Halving randomized search generate random parameter values following this strategy. The difference for Halving version is, just like for the halving grid version, the halving randomized search evaluates all hyper parameter combinations cheaply first to narrow down the more promising ones for more thorough evaluation (with more data).

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

param_distribs = [  # compare to the grid verson: following lines have distributions, not values.
    {'svm__C': loguniform(1e-3,1e+3),     # log(C) sampled from uniform distribution from -3 to 3
     'svm__gamma': loguniform(1e-2, 1e+2) # log(gamma) sampled from uniform distribution from -2 to 2
     },
    ]

random_search = RandomizedSearchCV(svm_pipeline, param_distribs, n_iter=20, cv=3,
                                 scoring='balanced_accuracy', random_state=42)
  # n_iter=20 specifies that 20 pair of (C, gamma) to be drawn. Evaluate them by cross validation.
  # Divide into K folds, keep 1 to test, fit 20 SVCs (with different C,gamma values) to training data
  # Rotate folds and repeat the process K times, then average the metric (balanced_accuracy here)
  #   across the folds for each of the 20 hyper-parameter combination.

random_search.fit(X_train, y_train)
random_search.best_estimator_
random_cv_res = pd.DataFrame(random_search.cv_results_)
random_cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
random_cv_res.filter(regex = '(^param_|mean_test_score)', axis=1).head()

,param_svm__C,param_svm__gamma,mean_test_score
1,24.658329,2.481041,0.641119
19,12.746712,0.576249,0.622752
4,4.042873,6.796578,0.622132
17,622.002598,17.123376,0.608117
6,98.777003,0.070690,0.602274


---
**Caution:**

A common suboptimal use of randomized search that we see is to supply the grid similar to that for grid search. While that'll "run", it simply uniformly randomly samples from the grid you specified. It doesn't explore the full space.

*Do not pass a grid into random search.* Use a random distribution.

---

Compared to the grid search, did you find better hyper-parameter values? That tends to happen because random search evaluates more values of each parameter.

Now the halving version:

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from scipy.stats import loguniform

param_distribs = [
    {'svm__C': loguniform(1e-3,1e+3),
     'svm__gamma': loguniform(1e-2, 1e+2)
     },
    ]
# This is where we switch to halving version ...
halving_random_search = HalvingRandomSearchCV(svm_pipeline, param_distribs,
                                      n_candidates=20, cv=3,
                                      min_resources='exhaust',
                                      scoring='balanced_accuracy',
                                      random_state=42)
halving_random_search.fit(X_train, y_train)
halving_random_cv_res = pd.DataFrame(halving_random_search.cv_results_)
# In the end, we care about performances in the last iteration (using most data)
# So, let's sort by iteration (descending), then by test score (descending)
halving_random_cv_res.sort_values(by=['iter', 'mean_test_score'], ascending=False, inplace=True)
# and check the top few rows
halving_random_cv_res.filter(regex = '(iter|^param_|mean_test_score|n_resources)', axis=1).head(10)
# halving_random_cv_res # to see the full table

,iter,n_resources,param_svm__C,param_svm__gamma,mean_test_score
29,2,7497,24.658329,2.481041,0.641118
28,2,7497,12.746712,0.576249,0.622751
27,2,7497,4.042873,6.796578,0.622131
25,1,2499,24.658329,2.481041,0.657373
26,1,2499,12.746712,0.576249,0.624379
24,1,2499,4.042873,6.796578,0.615621
22,1,2499,98.777003,0.070690,0.585566
23,1,2499,622.002598,17.123376,0.568786
21,1,2499,0.545029,13.826232,0.523604
20,1,2499,4.689401,0.036139,0.515873


You may have got the best parameters quicker. Thus, the advantage of the halving search (which grows with dataset size and number of parameters explored) is that using the same amount of time, you can explore more hyper parameter values.

Now that Randomized search has found a set of hyper-parameters that with best performance in our evaluations, we can use the corresponding model to measure prediction error on test data.

In [ ]:
from sklearn.metrics import balanced_accuracy_score

best_model = halving_random_search.best_estimator_  # access the best model, obtained by refitting
  # the model to the entire dataset after obtaining the best hyper-parameters
y_pred = best_model.predict(X_test) # predict on test data
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)  # evaluate predictions
print(f'Balanced accuracy of the best found model on the test data is {test_balanced_accuracy:.2f}')

Balanced accuracy of the best found model on the test data is 0.64


***
**Exercise**

(Team up with a neighbor)

Apply the search strategies to tune the following two hyper parameters of `KNeighborsClassifier`:
1. `n_neighbors`: The `k` in the k-nearest neighbors.
1. `p`: The norm of the distance used to find the `k` closest neighbors. I.e., $distance = (\sum_j^d ||x - y||^p)^{1/p}$. What does it mean? If you set p to:
  * 1: distance is City Block or Manhattan distance (sum of absolute differences across all components).
  * 2: distance is as crow flies (the standard geographic distance between two points along the most direct line)
  * ∞: distance is the max difference among components.
  
  The Eucledian distance may not be the best metric for all problems.

Note:
1. Both hyperparameters are integers. So, for grid search you can use `np.arange()` to pick a set of integers and for random search you can use `scipy.stats.randint` distribution to uniformly sample integers from a range.
1. Start with integers in the range of 2 to 10 (inclusive) spaced 2 units apart for Grid. Draw the same number of random integer parameter values for the random search strategies in the same range.

(It's fine to copy blocks of code from above and modify)

What did you find?
1. Any speed benefit of the Halving strategies on this dataset?
1. Any performance benefits of the Random strategies?

***

In [ ]:
# ============================================================
#  Exercise: KNN Hyperparameter Tuning with Search Strategies
# ============================================================

import numpy as np
import pandas as pd
from scipy.stats import randint
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import (
    GridSearchCV, RandomizedSearchCV,
    HalvingGridSearchCV, HalvingRandomSearchCV
)
from sklearn.experimental import enable_halving_search_cv
from sklearn.metrics import balanced_accuracy_score

# ---------------------------------------------
# 1️⃣ Define KNN pipeline (reuse preprocessing)
# ---------------------------------------------
knn_pipeline = Pipeline([
    ('preprocessing', preprocess_pipeline),   # reuse from your SVM code
    ('knn', KNeighborsClassifier())
])

# ---------------------------------------------
# 2️⃣ Define parameter ranges
# ---------------------------------------------
# For Grid Search — use fixed integer range
param_grid = {
    'knn__n_neighbors': np.arange(2, 11, 2),  # [2, 4, 6, 8, 10]
    'knn__p': np.arange(1, 4, 1)              # [1, 2, 3]
}

# For Random Search — use distributions for integer sampling
param_distribs = {
    'knn__n_neighbors': randint(2, 11),
    'knn__p': randint(1, 4)
}

# =====================================================
# 3️⃣ Grid Search
# =====================================================
grid_search = GridSearchCV(
    knn_pipeline,
    param_grid,
    cv=3,
    scoring='balanced_accuracy'
)
grid_search.fit(X_train, y_train)

grid_best = grid_search.best_estimator_
grid_pred = grid_best.predict(X_test)
grid_acc = balanced_accuracy_score(y_test, grid_pred)

print("---- Grid Search ----")
print("Best Params:", grid_search.best_params_)
print(f"CV Score: {grid_search.best_score_:.3f}")
print(f"Test Balanced Accuracy: {grid_acc:.3f}\n")

# =====================================================
# 4️⃣ Random Search
# =====================================================
random_search = RandomizedSearchCV(
    knn_pipeline,
    param_distributions=param_distribs,
    n_iter=5,
    cv=3,
    scoring='balanced_accuracy',
    random_state=42
)
random_search.fit(X_train, y_train)

rand_best = random_search.best_estimator_
rand_pred = rand_best.predict(X_test)
rand_acc = balanced_accuracy_score(y_test, rand_pred)

print("---- Random Search ----")
print("Best Params:", random_search.best_params_)
print(f"CV Score: {random_search.best_score_:.3f}")
print(f"Test Balanced Accuracy: {rand_acc:.3f}\n")

# =====================================================
# 5️⃣ Halving Grid Search
# =====================================================
halving_grid = HalvingGridSearchCV(
    knn_pipeline,
    param_grid,
    cv=3,
    scoring='balanced_accuracy',
    factor=2,
    min_resources='exhaust'
)
halving_grid.fit(X_train, y_train)

hgrid_best = halving_grid.best_estimator_
hgrid_pred = hgrid_best.predict(X_test)
hgrid_acc = balanced_accuracy_score(y_test, hgrid_pred)

print("---- Halving Grid Search ----")
print("Best Params:", halving_grid.best_params_)
print(f"CV Score: {halving_grid.best_score_:.3f}")
print(f"Test Balanced Accuracy: {hgrid_acc:.3f}\n")

# =====================================================
# 6️⃣ Halving Random Search
# =====================================================
halving_random = HalvingRandomSearchCV(
    knn_pipeline,
    param_distributions=param_distribs,
    n_candidates=5,
    cv=3,
    scoring='balanced_accuracy',
    min_resources='exhaust',
    random_state=42
)
halving_random.fit(X_train, y_train)

hrand_best = halving_random.best_estimator_
hrand_pred = hrand_best.predict(X_test)
hrand_acc = balanced_accuracy_score(y_test, hrand_pred)

print("---- Halving Random Search ----")
print("Best Params:", halving_random.best_params_)
print(f"CV Score: {halving_random.best_score_:.3f}")
print(f"Test Balanced Accuracy: {hrand_acc:.3f}\n")

# =====================================================
# 7️⃣ Compare results in summary table
# =====================================================
results_summary = pd.DataFrame({
    'Method': [
        'Grid Search',
        'Random Search',
        'Halving Grid',
        'Halving Random'
    ],
    'Best Params': [
        grid_search.best_params_,
        random_search.best_params_,
        halving_grid.best_params_,
        halving_random.best_params_,
    ],
    'CV Balanced Acc': [
        grid_search.best_score_,
        random_search.best_score_,
        halving_grid.best_score_,
        halving_random.best_score_,
    ],
    'Test Balanced Acc': [
        grid_acc,
        rand_acc,
        hgrid_acc,
        hrand_acc
    ]
})

print("\n================ Summary ================\n")
print(results_summary)
